In [361]:
import numpy as np

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling1D, MaxPooling2D, LeakyReLU
from keras.initializers import RandomNormal
import keras.optimizers as opt
from keras import regularizers
from keras.callbacks import TensorBoard
from keras.utils import plot_model

In [362]:
data = np.load('../data/data.npy')
meta = np.load('../data/meta.npy')
print(f'data = {data.shape}')
print(f'meta = {meta.shape}')

data = (1845, 21, 77)
meta = (1845, 3)


In [385]:
shape = X.shape
std = StandardScaler().fit(X.reshape(-1,1))
X = std.transform(X.reshape(-1,1)).reshape(shape)

In [386]:
X = data[:,:,:,np.newaxis]
Y = meta[:,2][:,np.newaxis].astype(int)

In [387]:
X,Y = shuffle(X,Y)

In [388]:
x_train, x_test,  y_train, y_test  = train_test_split(X, Y, test_size = 0.2, random_state = 31)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size = 0.25, random_state = 31)

In [389]:
print(f"train: x={x_train.shape} y={y_train.shape}")
print(f"valid: x={x_valid.shape}  y={y_valid.shape}")
print(f"test : x={x_test.shape}  y={y_test.shape}")

train: x=(1107, 21, 77, 1) y=(1107, 1)
valid: x=(369, 21, 77, 1)  y=(369, 1)
test : x=(369, 21, 77, 1)  y=(369, 1)


In [390]:
cnn_input = X.shape[1:]
cnn_output = Y.shape[1]
batch_size = 32

print(f"cnn_input  = {cnn_input}")
print(f"cnn_output = {cnn_output}")
print(f"batch_size = {batch_size}")

cnn_input  = (21, 77, 1)
cnn_output = 1
batch_size = 32


In [414]:
model = Sequential()

model.add(Conv2D(16, (7,7), activation = 'tanh', name = 'conv1', input_shape = cnn_input))
model.add(MaxPooling2D(pool_size=(2,2), strides=(1,1), name='maxpool1'))

model.add(Conv2D(32, (5,5), activation='tanh', name='conv2'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(1,1), name='maxpool2'))

model.add(Conv2D(64, (3,3), activation='tanh', name='conv3'))
model.add(MaxPooling2D(pool_size=(2,2), name='maxpool3'))

model.add(Flatten(name='flat4'))
model.add(Dropout(0.5, name='drop4'))
model.add(Dense(1024, activation='tanh', name='dense4'))

model.add(Dense(cnn_output, activation='sigmoid', name='softmax'))

In [415]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 15, 71, 16)        800       
_________________________________________________________________
maxpool1 (MaxPooling2D)      (None, 14, 70, 16)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 10, 66, 32)        12832     
_________________________________________________________________
maxpool2 (MaxPooling2D)      (None, 9, 65, 32)         0         
_________________________________________________________________
conv3 (Conv2D)               (None, 7, 63, 64)         18496     
_________________________________________________________________
maxpool3 (MaxPooling2D)      (None, 3, 31, 64)         0         
_________________________________________________________________
flat4 (Flatten)              (None, 5952)              0         
__________

In [416]:
optim = opt.SGD(lr = 0.001, decay = 1e-6, momentum = 0.5, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer=optim, metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, batch_size = batch_size, epochs = 100, 
          verbose = 2, validation_data=(x_valid, y_valid))

Train on 1107 samples, validate on 369 samples
Epoch 1/100
 - 2s - loss: 0.6269 - acc: 0.6703 - val_loss: 0.6039 - val_acc: 0.6612
Epoch 2/100
 - 0s - loss: 0.5711 - acc: 0.7136 - val_loss: 0.5691 - val_acc: 0.6667
Epoch 3/100
 - 0s - loss: 0.5441 - acc: 0.7290 - val_loss: 0.5401 - val_acc: 0.7209
Epoch 4/100
 - 0s - loss: 0.5179 - acc: 0.7516 - val_loss: 0.5108 - val_acc: 0.7453
Epoch 5/100
 - 0s - loss: 0.5020 - acc: 0.7597 - val_loss: 0.5291 - val_acc: 0.7290
Epoch 6/100
 - 0s - loss: 0.4785 - acc: 0.7805 - val_loss: 0.4791 - val_acc: 0.7642
Epoch 7/100
 - 0s - loss: 0.4567 - acc: 0.7967 - val_loss: 0.4990 - val_acc: 0.7615
Epoch 8/100
 - 0s - loss: 0.4422 - acc: 0.7904 - val_loss: 0.4623 - val_acc: 0.7751
Epoch 9/100
 - 0s - loss: 0.4294 - acc: 0.8121 - val_loss: 0.4522 - val_acc: 0.7913
Epoch 10/100
 - 0s - loss: 0.4210 - acc: 0.8094 - val_loss: 0.4334 - val_acc: 0.8157
Epoch 11/100
 - 0s - loss: 0.3995 - acc: 0.8257 - val_loss: 0.4139 - val_acc: 0.8103
Epoch 12/100
 - 0s - loss: 

In [411]:
for i in zip(model.metrics_names, model.evaluate(x_test, y_test, batch_size = 32)):
    print(i)

369/369 [==============================] - 0s 110us/step
('loss', 0.26156219339306114)
('acc', 0.9024390247133043)


In [412]:
model.predict(x_test)

array([[3.46353836e-03],
       [9.45678839e-05],
       [1.46563048e-03],
       [1.47280080e-05],
       [9.66611326e-01],
       [1.77613460e-03],
       [8.07347242e-05],
       [3.92729744e-05],
       [1.40072778e-03],
       [4.45781089e-02],
       [9.99965191e-01],
       [9.79811370e-01],
       [6.26313431e-06],
       [8.76022968e-04],
       [1.26831088e-04],
       [7.90224552e-01],
       [1.95097920e-04],
       [2.13064704e-05],
       [2.28710269e-04],
       [9.96271014e-01],
       [1.52214943e-05],
       [7.22936820e-03],
       [1.74941728e-03],
       [7.94359803e-01],
       [6.42049536e-02],
       [9.94116664e-01],
       [9.87021863e-01],
       [1.60466582e-01],
       [5.50831246e-05],
       [1.94129755e-03],
       [6.89038716e-04],
       [1.51928350e-01],
       [2.27636516e-01],
       [9.99931812e-01],
       [7.14440405e-01],
       [9.99933958e-01],
       [4.83234264e-02],
       [4.36295150e-03],
       [9.99348342e-01],
       [2.94642955e-01],
